In [1]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
import pandas as pd
import matplotlib.animation as animation
from matplotlib.colors import LightSource
from matplotlib import cm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
## Function to compute exact grounding line position from f_grnd

def find_exact_GL_position(ds_hf, y_coord, add_to_xarray=False, save_to_netcdf=False, savepath=None):

    # Create empty array, same length as time array
    GL_position = np.zeros([len(ds_hf.time.values)])
    
    # Iterate over all time steps in output
    for i,t in enumerate(ds_hf.time.values):
        
        # For first time step, the f_grnd variable is not computed. Use the last time step of spinup instead.
        if i == 0:
            GL_position[i] = 51004.59816957 # initial grounding line from spinup
        
        # For other time steps, select the right y coordinate and locate at which location the value of f_grnd is > 0 but < 1
        else:
            ds_hf_fgrnd_y = ds_hf.sel(y=y_coord, time=t).f_grnd
            ds_hf_fgrnd_y = ds_hf_fgrnd_y.where(np.logical_and(ds_hf_fgrnd_y>0,ds_hf_fgrnd_y<1), drop=True)
            # If there are two values for which the above condition is true, set fraction to 1 (safety)
            if len(ds_hf_fgrnd_y.values)>1:
                frac = 1
            else:
                try:
                    frac = ds_hf_fgrnd_y.values[0]
                except:
                    frac = 1
            # Compute the GL_position based on the floating fraction. When 90% of the cell is afloat --> GL position is the x value at which 0 < f_grnd < 1 + 10% of the grid cell length 
            try:
                GL_pos_y = frac * 2000 + ds_hf_fgrnd_y.x.values[0]
            except: 
                print('use previous GL_pos_y')            
            GL_position[i] = GL_pos_y

    # Add to the file if wanted
    if add_to_xarray == True:
        ds_hf = ds_hf.assign(GLpos=(('time',), GL_position))
        ds_GL = ds_hf['GLpos']

        # Save to netcdf file if wanted
        if save_to_netcdf == True:
            ds_GL.to_netcdf(savepath)

    return ds_hf, GL_position

## Post process

In [ ]:
# Post processing loop for MAIN runs and RETUNED runs
dir = '/Volumes/T7/P1_DIR/Model_output/NOcalving/' #'/Users/5941962/surfdrive/P1_DIR/Model_output'
exp_names = [ 

    #MAIN RUNS
    'P1_MEDIUM_QUAD_NOcalving',
    'P1_MEDIUM_PICO_NOcalving_C023', 
    'P1_MEDIUM_PLUME_NOcalving',
    'P1_MEDIUM_laddie_NOcalving_rhoice_concat',

    'P1_WARM_QUAD_NOcalving_modtune',
    'P1_WARM_PICO_NOcalving_C023_modtune', 
    'P1_WARM_PLUME_NOcalving_modtune',
    'P1_WARM_laddie_NOcalving_rhoice_concat', 

    #RETUNED RUNS
    'P1_MEDIUM_QUAD_NOcalving_hightune', 
    'P1_MEDIUM_PICO_NOcalving_C023_hightune', 
    'P1_MEDIUM_PLUME_NOcalving_hightune', 

    'P1_WARM_QUAD_NOcalving',
    'P1_WARM_PICO_NOcalving_C023', 
    'P1_WARM_PLUME_NOcalving',
]

filename = 'help_fields_ANT.nc'
combined_list = [f'{dir}/{exp_name}/{filename}' for exp_name in exp_names]


for i in range(len(combined_list)):

    print(f'Started computation GL position for {exp_names[i]} ...')

    # Open file
    ds = xr.open_dataset(f'{dir}/{exp_names[i]}/{filename}')

    # Compute exact grounding line position from f_grnd
    find_exact_GL_position(ds, y_coord=0, add_to_xarray=True, save_to_netcdf=True, savepath=f'../PostProcess/GL_{exp_names[i]}.nc')

    print(f'... DONE')
    ds.close()

In [ ]:
# Post processing loop for CONTROL & CALVING runs
dir = '/Volumes/T7/P1_DIR/Model_output/' #'/Users/5941962/surfdrive/P1_DIR/Model_output'
exp_names = [ 
    #CONTROL RUN
    'P1_control',
    
    #CALVING
    'P1_WARM_QUAD_modtune',
    'P1_WARM_PICO_C023_modtune', 
    'P1_WARM_PLUME_modtune',
    'P1_WARM_laddie_rhoice_concat', 

    'P1_MEDIUM_QUAD',
    'P1_MEDIUM_PICO_C023', 
    'P1_MEDIUM_PLUME',
    'P1_MEDIUM_laddie_rhoice_concat',
]
filename = 'help_fields_ANT.nc'
combined_list = [f'{dir}/{exp_name}/{filename}' for exp_name in exp_names]


for i in range(len(combined_list)):

    print(f'Started computation GL position for {exp_names[i]} ...')

    # Open file
    ds = xr.open_dataset(f'{dir}/{exp_names[i]}/{filename}')

    # Compute exact grounding line position from f_grnd
    find_exact_GL_position(ds, y_coord=0, add_to_xarray=True, save_to_netcdf=True, savepath=f'../PostProcess/GL_{exp_names[i]}.nc')

    print(f'... DONE')
    ds.close()